In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [13]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
#PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release_20150715/bin'
PROGRAM_PATHS = '/scratch/BS/release-20160519/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from SEMTools import *

In [14]:
#DWI_scan = '/Shared/johnsonhj/TrackOn/Experiments/20140203_DWIProcessing/VISUALLY_INSPECTED/HDNI_003/823955340/823955340_20080609_30/823955340_823955340_20080609_30_DWI_CONCAT_QCed_INSPECTED.nrrd'
DWI_scan = '/Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/105115/T1w/Diffusion/data.nrrd'
DWI=sitk.ReadImage(DWI_scan)

In [19]:
print(DWI.GetDirection())

(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [15]:
rigTx = sitk.VersorRigid3DTransform()
rigTx.SetRotation([0,0,1],180)
sitk.WriteTransform(rigTx,'/scratch/TESTS/tmp/rigTx.h5')
#print(rigTx)

In [16]:
#def GetTransfromInverse(inputTransform):
#    import SimpleITK as sitk
#    inverseTransform = sitk.VersorRigid3DTransform()
#    inverseTransform.SetFixedParameters(inputTransform.GetFixedParameters())
#    inverseTransform.SetParameters(inputTransform.GetParameters())
#    inverseTransform.SetInverse()
#    return inverseTransform

In [17]:
#invTx = GetTransfromInverse(rigTx)
#sitk.WriteTransform(invTx,'/scratch/TESTS/tmp/inverseRigTx.h5')
#print(invTx)

In [18]:
gtRIP=gtractResampleDWIInPlace()
gtRIP.inputs.inputVolume = DWI_scan
gtRIP.inputs.inputTransform = '/scratch/TESTS/tmp/rigTx.h5'
gtRIP.inputs.outputVolume = '/scratch/TESTS/tmp/rotatedDWI.nrrd'
gtRIP.run()

INFO:interface:stdout 2016-05-20T13:29:25.771632:=====================================================
INFO:interface:stdout 2016-05-20T13:29:25.771632:DWI Image: /Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/105115/T1w/Diffusion/data.nrrd
INFO:interface:stdout 2016-05-20T13:29:25.771632:Input Transform: /scratch/TESTS/tmp/rigTx.h5
INFO:interface:stdout 2016-05-20T13:29:25.771632:warpDWI Transform: 
INFO:interface:stdout 2016-05-20T13:29:25.771632:Output Image: /scratch/TESTS/tmp/rotatedDWI.nrrd
INFO:interface:stdout 2016-05-20T13:29:25.771632:Debug Level: 0
INFO:interface:stdout 2016-05-20T13:29:25.771632:Image Output Size: 0,0,0
INFO:interface:stdout 2016-05-20T13:29:25.771632:=====================================================
INFO:interface:stdout 2016-05-20T13:29:39.410710:Read Image
INFO:interface:stdout 2016-05-20T13:29:39.432620:Read ITK transform from file: /scratch/TESTS/tmp/rigTx.h5
INFO:interface:stdout 2016-05-20T13:29:39.966860:HACK: 0  VersorRigid3DTransform
INFO:inter

In [61]:
#gtRIP2=gtractResampleDWIInPlace()
#gtRIP2.inputs.inputVolume = gtRIP.inputs.outputVolume
#gtRIP2.inputs.inputTransform = '/scratch/TESTS/tmp/inverseRigTx.h5'
#gtRIP2.inputs.outputVolume = '/scratch/TESTS/tmp/restoredDWI.nrrd'
#gtRIP2.run()

In [64]:
DTIEstim=dtiestim()
DTIEstim.inputs.method = 'wls'
DTIEstim.inputs.tensor_output = '/scratch/TESTS/tmp/original_DTI.nrrd'
DTIEstim.inputs.dwi_image = DWI_scan
DTIEstim.run()

In [65]:
DTIEstim2=dtiestim()
DTIEstim2.inputs.method = 'wls'
DTIEstim2.inputs.tensor_output = '/scratch/TESTS/tmp/rotated_DTI.nrrd'
DTIEstim2.inputs.dwi_image = gtRIP.inputs.outputVolume
DTIEstim2.run()